In [1]:
import re
import time
import pymysql
import pandas as pd
import numpy as np
from gensim.models import word2vec

E:\Anaconda\envs\clien\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 데이터베이스 연결

In [4]:
db = pymysql.connect(\
    host="172.29.235.145",\
    port=3306,\
    user="root",\
    passwd="19900404",\
    charset='utf8mb4'\
)

### 데이터 읽기 및 확인

In [5]:
# SQL = "SELECT * FROM clien.mogong_contents where time > '2018-12-31'"
SQL = "SELECT * FROM clien.mogong_contents"
df = pd.read_sql(SQL, db)

In [6]:
df.head()

,id,nickname,ip,symph,hit,title,contents,pic_num,comm_num,time,updated_time,latest_time,url
0,11606811,마야준님,1.♡.231.233,0,749.0,2018 새해 복 많이 받으세여^^,새해 첫글이. 되길...,0,1.0,2017-12-31 23:59:59,None,2017-12-31 23:59:59,https://www.clien.net/service/board/park/11606...
1,11606812,None,None,0,NaN,None,None,0,NaN,None,None,None,https://www.clien.net/service/board/park/11606...
2,11606813,뺈쎬,120.♡.110.8,0,NaN,2018.1.1,어그로,0,22.0,2018-01-01 00:00:00,None,2018-01-01 00:00:00,https://www.clien.net/service/board/park/11606...
3,11606814,롱팔이,222.♡.82.18,0,912.0,happy new year,2018년에는 모두에게 좋은일만 있기를..,0,17.0,2018-01-01 00:00:00,None,2018-01-01 00:00:00,https://www.clien.net/service/board/park/11606...
4,11606815,스프라이트샤워,112.♡.126.2,0,326.0,펑,펑,0,0.0,2018-01-01 00:00:00,None,2018-01-01 00:00:00,https://www.clien.net/service/board/park/11606...


### 데이터 전처리 및 저장

In [7]:
file = "./data/clien_monong_content.txt"
writeFp = open(file, "w", encoding="utf-8")

In [8]:
def simple_parser(txt):
    txt = re.sub(r'\n', ' ', txt)
    txt = re.sub('[A-Z]', ' ', txt)
    txt = re.sub('[a-z]', ' ', txt)
    txt = re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.split(" ")
    return txt

In [9]:
contents = [content for content in df.contents if content!=None]

In [10]:
# 텍스트를 한 줄씩 처리하기
i = 0
start_time = time.time()
for content in contents:
    word_list = simple_parser(content)
    if i % 1000 == 0:
        print("current - " + str(i))
    i=i+1
    for word in word_list:
        writeFp.write(word + " ")

writeFp.close()
time_taken = time.time()-start_time
print(str(time_taken))

current - 0
current - 1000
current - 2000
current - 3000
current - 4000
current - 5000
current - 6000
current - 7000
current - 8000
current - 9000
current - 10000
current - 11000
current - 12000
current - 13000
current - 14000
current - 15000
current - 16000
current - 17000
current - 18000
current - 19000
current - 20000
current - 21000
current - 22000
current - 23000
current - 24000
current - 25000
current - 26000
current - 27000
current - 28000
current - 29000
current - 30000
current - 31000
current - 32000
current - 33000
current - 34000
current - 35000
current - 36000
current - 37000
current - 38000
current - 39000
current - 40000
current - 41000
current - 42000
current - 43000
current - 44000
current - 45000
current - 46000
current - 47000
current - 48000
current - 49000
current - 50000
current - 51000
current - 52000
current - 53000
current - 54000
current - 55000
current - 56000
current - 57000
current - 58000
current - 59000
current - 60000
current - 61000
current - 62000
curre

In [11]:
data = word2vec.LineSentence(file)
model = word2vec.Word2Vec(data, size=100, window=5, hs=1, min_count=2, sg=1)
model.save("clien_monong.model")
print("ok")

ok


In [12]:
model = word2vec.Word2Vec.load("clien_monong.model")

In [13]:
model.most_similar(positive=["삼성"])

E:\Anaconda\envs\clien\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('구매하시겠습니까', 0.8118062019348145),
 ('입간판도', 0.8031802177429199),
 ('불매', 0.7984183430671692),
 ('완제품', 0.782915472984314),
 ('그래픽드라이버를', 0.7767780423164368),
 ('가입하시던데', 0.7584606409072876),
 ('생명이나', 0.7540336847305298),
 ('삼성전자', 0.7495864629745483),
 ('백색가전', 0.7488430738449097),
 ('할인률', 0.7453415989875793)]

In [14]:
model.wv.most_similar(positive=["애플"])

[('아이폰', 0.8748095035552979),
 ('애플이', 0.8343870639801025),
 ('화웨이', 0.8268133401870728),
 ('제품', 0.8226539492607117),
 ('갤럭시', 0.818682074546814),
 ('신형', 0.8108293414115906),
 ('모델', 0.8058336973190308),
 ('최신', 0.8037415742874146),
 ('쓰고싶을땐', 0.7960079908370972),
 ('스마트폰', 0.7930388450622559)]